### Importing necessary packages

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

# Model Architecture

## EncoderDecoder

In [9]:
"""
This is a classic encoder decoder architecture
"""
class EncoderDecoder(nn.Module):
    """
    This class inherits from the nn.module the base class for all neural network modules.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.generator = generator
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        """
        Take in and process masked src and target sequences
        """
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src),src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

## Generator

In [ ]:
class Generator(nn.Module):
    """
    Define standard linear + softmax generation steps
    """
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    def forward(self, x): # x is (d_model, vocab)
        return F.log_softmax(self.proj(x), dim=-1)

## Encoder

In [10]:
def clones(module, N):
    """
    Produce N identical layers
    """
class Encoder(nn.Module):
    """
    Core encoder is a stack of N layers
    """
    def __init__(self, layer, N):
        super(Encoder,self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size) # We will build the class layernorm
    
    def forward(self, x, mask):
        """
        Pass the input (and mask) through each layer in turn
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

## LayerNorm

In [ ]:
class LayerNorm(nn.Module):
    """
    Employ a residual connection around each of the two sub-layers, 
    followed by layer normalization.
    
    Deeper neural networks are more difficult to train. 
    We present a residual learning framework to ease the training of networks 
    that are substantially deeper than those used previously. 
    We explicitly reformulate the layers as learning residual functions 
    with reference to the layer inputs, instead of learning unreferenced functions. 
    We provide comprehensive empirical evidence showing that these residual networks 
    are easier to optimize, and can gain accuracy from considerably increased depth. 
    On the ImageNet dataset we evaluate residual nets with a depth of up to 
    152 layers---8x deeper than VGG nets but still having lower complexity. 
    An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. 
    This result won the 1st place on the ILSVRC 2015 classification task. 
    We also present analysis on CIFAR-10 with 100 and 1000 layers.
    The depth of representations is of central importance for many visual recognition tasks. 
    Solely due to our extremely deep representations, we obtain a 28% relative improvement 
    on the COCO object detection dataset. 
    Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions, 
    where we also won the 1st places on the tasks of ImageNet detection, 
    ImageNet localization, COCO detection, and COCO segmentation.
    
    Construct a layernorm module:
    """
    def __init__(self, features, eps=1e-6): # features = layer.size
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    def forward(self, x):
        mean = x.mean(-1, keepdim = True)
        std = x.std(-1, keepdim = True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2